In [14]:
import pandas as pd
import numpy as np

bios = pd.read_excel("./data/olympics-data.xlsx")

coffee = pd.read_csv("tabelaCafé.csv")

nocs = pd.read_csv("./data/noc_regions.csv")

def categorize_athlete(row):
    if row["height_cm"] < 175 and row["weight_kg"] < 70:
        return "Light Weight"
    elif row["height_cm"] < 185 or row["weight_kg"] <= 80:
        return "Middle Weight"
    
    else:
        return "Heavy Weight"

bios["Category"] = bios.apply(categorize_athlete, axis=1) #axis=1 são as linhas; axis=0 colunas

results = pd.read_parquet("./data/results.parquet")

#print(results.columns)

#display(bios[(bios["height_cm"] > 215) & (bios["born_country"]=="USA")][["name", "height_cm"]])

#bios_filtered1 = bios.query('name.str.contains("Patrick|Keith", case=False, na=False)', engine='python')

#bios_filtered2 = bios.query('born_country in ["USA", "GBR"] and name.str.contains("Patrick|Keith", case=False, na=False)', engine='python')

#bios_filtered3 = bios.query('born_country in ["USA"]')


#display(bios_filtered1)
#display(bios_filtered2)
#display(bios_filtered3)

#coffee["OldPrice"] = 7.00
#(coffee)
#.drop(columns=["OldPrice"], inplace=True) #inplace=True serve para modificar de fato o arquivo da tabela

coffee["price"] = np.where(coffee["Coffee Type"]=="Espresso", 7.00, 10.00)
coffee["Revenue"] = coffee["Units Sold"] * coffee["price"]

coffee = coffee.rename(columns={"price":"Price"})

coffee = coffee.dropna(subset=["Units Sold"]) # retira as rows apenas nas quais a coluna Units Sold nn tiver nada - 
                                              # inplace=True retorna None e modifica o Dataframe original. TOMAR CUIDADO

coffee_new = coffee.copy() #copia de fato o conteúdo, não somente o mesmo espaço de memória

display(coffee)

bios_new = bios.copy()

bios_new["first_name"] = bios_new["name"].str.split(" ").str[0]
#print(bios_new["first_name"].head(10))

#bios_new.query('first_name.str.contains("Patrick|Keith", case=False, na=False)', engine="python")
#bios_new.query('first_name == "Patrick" and born_country in ["USA", "GBR"]')

#bios_filtered4 = bios_new.query('first_name in ["Patrick", "Keith"]')

bios_new["borndt"] = pd.to_datetime(bios_new["born_date"])
bios_new["born_dotw"] = bios_new["borndt"].dt.day_of_week # Monday = 0 e Sunday = 
bios_new["born_year"] = bios_new["borndt"].dt.year

nascimentos_por_ano = bios_new.groupby(bios_new["born_year"])["name"].count().reset_index().sort_values("name", ascending=False)
display(nascimentos_por_ano)

display(bios_new[["first_name", "born_year"]]) 

bios_new.to_csv("./data/bios_new.csv")

bios["height_category"] = bios["height_cm"].apply(lambda x: "Short" if x < 165 else ("Average" if x < 185 else "Tall"))

bios_new = pd.merge(bios, nocs, left_on="born_country", right_on="NOC", how="left", suffixes=["bios", "noc"])
bios_new.rename(columns={"region": "born_country_full"}, inplace=True)

bios_new = bios_new[bios_new["NOCbios"] != bios_new["born_country_full"]][["name", "NOCbios", "born_country_full"]]

bra = bios[bios["born_country"]=="BRA"].copy()
gbr = bios[bios["born_country"]=="GBR"].copy()

display(pd.concat([bra, gbr]))

combined_df = pd.merge(results, bios, on="athlete_id", how="left")

display(combined_df)

bios["born_country"].value_counts()
bios_bra = bios[bios["born_country"]=="BRA"]["born_region"].value_counts()
display(bios_bra)

#results.groupby(results["discipline"])["discipline"].value_counts() MODIFICAR
print(coffee_new.columns)
coffee_grouped = coffee_new.groupby(["Coffee Type"]).agg({"Units Sold": "sum", "Price": "mean"})

pivot = coffee.pivot(columns="Coffee Type", index="Day", values="Revenue")

coffee["yesterday_revenue"] = coffee["Revenue"].shift(2) #Shift compara com os valores anteriores
coffee["percentage_change"] = coffee["Revenue"] / coffee["yesterday_revenue"] *  100 

bios["height_rank"] = bios["height_cm"].rank(ascending=False)

display(coffee)
display(coffee_grouped)
display(pivot)
display(bios.sort_values(["height_rank"]).sample(10)[["name", "height_rank"]])

#display(bios_new)
#display(bios_filtered4)

,Day,Coffee Type,Units Sold,Price,Revenue
0,Monday,Espresso,25,7.0,175.0
1,Monday,Latte,15,10.0,150.0
2,Tuesday,Espresso,30,7.0,210.0
3,Tuesday,Latte,20,10.0,200.0
4,Wednesday,Espresso,35,7.0,245.0
5,Wednesday,Latte,25,10.0,250.0
6,Thursday,Espresso,40,7.0,280.0
7,Thursday,Latte,30,10.0,300.0
8,Friday,Espresso,45,7.0,315.0
9,Friday,Latte,35,10.0,350.0


,born_year,name
139,1972.0,2231
152,1985.0,2227
140,1973.0,2216
138,1971.0,2205
137,1970.0,2174
...,...,...
4,1837.0,1
2,1833.0,1
6,1839.0,1
12,1845.0,1


,first_name,born_year
0,Jean-François,1886.0
1,Arnaud,1969.0
2,Jean,1898.0
3,Jacques,1895.0
4,Albert,1878.0
...,...,...
145495,Polina,2002.0
145496,Valeriya,1999.0
145497,Yuliya,1998.0
145498,André,1899.0


,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date,Category,height_category
143,144,Marc-Kevin Goellner,1970-09-22,Rio de Janeiro,Rio de Janeiro,BRA,Germany,196.0,85.0,NaN,Heavy Weight,Tall
353,354,Arci Kempner,1934-02-03,Porto Alegre,Rio Grande do Sul,BRA,Brazil,170.0,60.0,NaN,Light Weight,Average
354,355,Jorge Azevedo,1950-05-10,Rio de Janeiro,Rio de Janeiro,BRA,Brazil,168.0,70.0,NaN,Middle Weight,Average
355,356,Renato Dutra e Mello,1955-05-12,Rio de Janeiro,Rio de Janeiro,BRA,Brazil,167.0,72.0,NaN,Middle Weight,Average
356,357,Vítor Krieger,1960-04-30,Porto Alegre,Rio Grande do Sul,BRA,Brazil,NaN,NaN,NaN,Heavy Weight,Tall
...,...,...,...,...,...,...,...,...,...,...,...,...
144811,148512,Benjamin Alexander,1983-05-08,London,England,GBR,Jamaica,NaN,NaN,NaN,Heavy Weight,Tall
144815,148517,Ashley Watson,1993-10-28,Peterborough,England,GBR,Jamaica,NaN,NaN,NaN,Heavy Weight,Tall
145005,148716,Peder Kongshaug,2001-08-13,Wimbledon,England,GBR,Norway,184.0,86.0,NaN,Middle Weight,Average
145319,149041,Axel Brown,1992-04-02,Harrogate,England,GBR,Trinidad and Tobago,NaN,NaN,NaN,Heavy Weight,Tall


,year,type,discipline,event,as,athlete_id,noc,team,place,tied,...,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date,Category,height_category
0,1912.0,Summer,Tennis,"Singles, Men (Olympic)",Jean-François Blanchy,1,FRA,None,17.0,True,...,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Heavy Weight,Tall
1,1912.0,Summer,Tennis,"Doubles, Men (Olympic)",Jean-François Blanchy,1,FRA,Jean Montariol,NaN,False,...,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Heavy Weight,Tall
2,1920.0,Summer,Tennis,"Singles, Men (Olympic)",Jean-François Blanchy,1,FRA,None,32.0,True,...,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Heavy Weight,Tall
3,1920.0,Summer,Tennis,"Doubles, Mixed (Olympic)",Jean-François Blanchy,1,FRA,Jeanne Vaussard,8.0,True,...,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Heavy Weight,Tall
4,1920.0,Summer,Tennis,"Doubles, Men (Olympic)",Jean-François Blanchy,1,FRA,Jacques Brugnon,4.0,False,...,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Heavy Weight,Tall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308403,2022.0,Winter,Luge,"Singles, Men (Olympic)",Marián Skupek,148983,SVK,None,26.0,False,...,2001-07-12,Gelnica,Košice,SVK,Slovakia,196.0,108.0,NaN,Heavy Weight,Tall
308404,2022.0,Winter,Alpine Skiing (Skiing),"Slalom, Women (Olympic)",Elsa Fermbäck,148984,SWE,None,28.0,False,...,1998-03-28,Vemdalen,Jämtland,SWE,Sweden,NaN,NaN,NaN,Heavy Weight,Tall
308405,2022.0,Winter,Alpine Skiing (Skiing),"Team, Mixed (Olympic)",Hilma Lövblom,148985,SWE,Sweden,13.0,False,...,2000-08-16,Täby,Stockholm,SWE,Sweden,NaN,NaN,NaN,Heavy Weight,Tall
308406,2022.0,Winter,Alpine Skiing (Skiing),"Giant Slalom, Women (Olympic)",Hilma Lövblom,148985,SWE,None,NaN,False,...,2000-08-16,Täby,Stockholm,SWE,Sweden,NaN,NaN,NaN,Heavy Weight,Tall


...

Index(['Day', 'Coffee Type', 'Units Sold', 'Price', 'Revenue'], dtype='object')


,Day,Coffee Type,Units Sold,Price,Revenue,yesterday_revenue,percentage_change
0,Monday,Espresso,25,7.0,175.0,NaN,NaN
1,Monday,Latte,15,10.0,150.0,NaN,NaN
2,Tuesday,Espresso,30,7.0,210.0,175.0,120.000000
3,Tuesday,Latte,20,10.0,200.0,150.0,133.333333
4,Wednesday,Espresso,35,7.0,245.0,210.0,116.666667
5,Wednesday,Latte,25,10.0,250.0,200.0,125.000000
6,Thursday,Espresso,40,7.0,280.0,245.0,114.285714
7,Thursday,Latte,30,10.0,300.0,250.0,120.000000
8,Friday,Espresso,45,7.0,315.0,280.0,112.500000
9,Friday,Latte,35,10.0,350.0,300.0,116.666667


,Units Sold,Price
Coffee Type,,
Espresso,265,7.0
Latte,195,10.0


Coffee Type,Espresso,Latte
Day,,
Friday,315.0,350.0
Monday,175.0,150.0
Saturaday,315.0,350.0
Sunday,315.0,350.0
Thursday,280.0,300.0
Tuesday,210.0,200.0
Wednesday,245.0,250.0


,name,height_rank
34866,Alberto Ortiz,NaN
25112,Bernd Bauchspieß,31471.5
77614,Bill Carr,69980.5
26060,Kim Jeong-Seok,54014.0
123505,Ellen Hogerwerf,27597.5
46926,Dave Haller,24214.0
65222,Mihail Bachvarov,62600.5
112338,Hamed Afagh,13590.5
91405,Marek Marcińczak,46966.0
26639,Kristian Krefting,NaN
